In [1]:
import os
import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from func import *
import json
import ast
from skimage import feature

In [2]:
df = pd.read_csv("merged_df.csv")
df

,image_id,bbox_id,keypoints,center,scale,area,score,bbox,width,height,bbox_size,x,y
0,139,0,"[[429.280517578125, 169.61883544921875, 0.8657...","[438.82501220703125, 226.11500549316406]","[0.6422343850135803, 0.8563125133514404]",21998.134766,0.763357,"[412.8, 157.61, 53.05, 138.01]",53.05,138.01,7321.4305,412.80,157.61
1,785,0,"[[367.1922302246094, 80.87351989746094, 0.9483...","[389.6400146484375, 217.57000732421875]","[1.6203750371932983, 2.1605000495910645]",140032.812500,0.902015,"[280.79, 44.73, 218.7, 346.68]",218.70,346.68,75818.9160,280.79,44.73
2,872,0,"[[214.49267578125, 184.88751220703125, 0.79049...","[290.7349853515625, 328.8450012207031]","[2.1391406059265137, 2.852187395095825]",244049.203125,0.664442,"[145.26, 100.67, 291.95, 457.35]",291.95,457.35,133523.3325,145.26,100.67
3,885,2,"[[632.353759765625, 46.84330749511719, 0.96756...","[616.864990234375, 139.02000427246094]","[1.0616250038146973, 1.4155000448226929]",60109.207031,0.805771,"[595.68, 25.78, 43.37, 227.48]",43.37,227.48,9865.8076,595.68,25.78
4,885,0,"[[373.1753234863281, 217.2002716064453, 0.9457...","[346.8550109863281, 293.6000061035156]","[0.9713437557220459, 1.2951250076293945]",50320.460938,0.843057,"[277.31, 189.99, 140.09, 208.22]",140.09,208.22,29169.5398,277.31,189.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,580294,0,"[[587.7160034179688, 73.6966552734375, 0.31208...","[525.0700073242188, 220.07000732421875]","[2.034562587738037, 2.712750196456909]",220770.406250,0.383998,"[411.14, 3.05, 228.86, 435.04]",228.86,435.04,99563.2544,411.14,3.05
6330,581062,0,"[[163.4222869873047, 66.69705200195312, 0.9830...","[162.58999633789062, 107.5250015258789]","[0.5272969007492065, 0.7030624747276306]",14828.906250,0.896881,"[131.74, 51.28, 62.7, 113.49]",62.70,113.49,7115.8230,131.74,51.28
6331,581206,0,"[[347.857177734375, 109.86737060546875, 0.1125...","[234.26499938964844, 109.66000366210938]","[2.928312301635742, 3.904416561126709]",457334.062500,0.339190,"[0, 2.53, 469.53, 215.26]",469.53,215.26,101071.0278,0.00,2.53
6332,581317,0,"[[501.866455078125, 118.10910034179688, 0.9580...","[511.06500244140625, 204.46499633789062]","[1.3925156593322754, 1.8566875457763672]",103418.664062,0.849606,"[409.85, 55.93, 203.43, 298.07]",203.43,298.07,60636.3801,409.85,55.93


In [3]:
df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'area', 'score',
       'bbox', 'width', 'height', 'bbox_size', 'x', 'y'],
      dtype='object')

In [4]:
df.drop(columns=['bbox_id','keypoints','area'], inplace=True)

In [5]:
df["image_id"] = df["image_id"].astype(str).apply(lambda x: x.rjust(12, '0') + ".jpg")
df['bbox'] = df['bbox'].apply(ast.literal_eval)

In [6]:
df['image_file'] = str('val2017/')+df['image_id']
df.drop(columns='image_id', inplace=True)

In [7]:
df.columns

Index(['center', 'scale', 'score', 'bbox', 'width', 'height', 'bbox_size', 'x',
       'y', 'image_file'],
      dtype='object')

## metric 계산 적용

In [8]:
entropy = []
laplacian = []
brightness = []
B_sd = []
c_moment = []
contra = []
img_size = []
edge_intensity = []
edge_num = []
sift = []

for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    height, width= image.shape
    img_size_value = width * height

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    entropy_value = calculate_image_entropy_(cropped_image)
    laplacian_value = calculate_laplacian_variance_(cropped_image)
    brightness_value = np.mean(cropped_image)
    sd_value = np.std(cropped_image)
    c_moment_value = moment(cropped_image)
    contrast_value = calculate_contrast(cropped_image)
    edge_num_value, edge_value = edge(cropped_image)
    SIFT_count_value = SIFT_count(cropped_image)

    entropy.append(entropy_value)
    laplacian.append(laplacian_value)
    brightness.append(brightness_value)
    B_sd.append(sd_value)
    c_moment.append(c_moment_value)
    contra.append(contrast_value)
    img_size.append(img_size_value)
    edge_intensity.append(edge_value)
    edge_num.append(edge_num_value)
    sift.append(SIFT_count_value)

In [9]:
red = []
green = []
blue = []
color_var = []

# Not Grayscale
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    
    # 이미지 배열에서 각 채널의 값 가져오기
    red_channel = cropped_image[:, :, 0]
    green_channel = cropped_image[:, :, 1]
    blue_channel = cropped_image[:, :, 2]

    # 각 채널별 평균과 분산 계산
    red_mean, red_variance = np.mean(red_channel), np.var(red_channel)
    green_mean, green_variance = np.mean(green_channel), np.var(green_channel)
    blue_mean, blue_variance = np.mean(blue_channel), np.var(blue_channel)
    variance = np.mean([red_variance,green_variance,blue_variance])

    red.append(red_mean)
    green.append(green_mean)
    blue.append(blue_mean)
    color_var.append(variance)

In [10]:
saturation = []

# Not Grayscale
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]

    saturation_value = np.mean(cropped_image[:, :, 1])

    saturation.append(saturation_value)

In [11]:
df['entropy'] = entropy
df['laplacian'] = laplacian
df['brightness'] = brightness
df['B_sd'] = B_sd
df['red'] = red
df['blue'] = blue
df['green'] = green
df['color_var'] = color_var
df['c_moment'] = c_moment
df['contrast'] = contra
df['img_size'] = img_size
df['edge_intensity'] = edge_intensity
df['edge_num'] = edge_num
df['saturation'] = saturation
df['sift'] = sift

In [12]:
# 새로운 열 생성
df[['c_moment_x', 'c_moment_y']] = pd.DataFrame(df['c_moment'].tolist(), index=df.index)

# 기존 열 삭제
# df = df.drop('c_moment', axis=1)

In [13]:
df['bbox_prop'] = df['bbox_size']/df['img_size'].values

In [14]:
df = df[['image_file'] + [col for col in df.columns if col != 'image_file']]

In [15]:
tmp = df.sort_values(by='score', ascending=False).reset_index(drop=True)

In [16]:
tmp.columns

Index(['image_file', 'center', 'scale', 'score', 'bbox', 'width', 'height',
       'bbox_size', 'x', 'y', 'entropy', 'laplacian', 'brightness', 'B_sd',
       'red', 'blue', 'green', 'color_var', 'c_moment', 'contrast', 'img_size',
       'edge_intensity', 'edge_num', 'saturation', 'sift', 'c_moment_x',
       'c_moment_y', 'bbox_prop'],
      dtype='object')

In [17]:
tmp.to_csv("refine_merged_df.csv", index=False)

-----------

### Haralick feature

In [18]:
import mahotas

image_file = []
haralick = []
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]

    # Haralick 텍스처 특징 계산
    features = mahotas.features.haralick(cropped_image).mean(axis=0)
    haralick.append(features)
    image_file.append(image_file_path)

In [19]:
# 각 Haralick 특징을 열로 하는 데이터프레임 생성
haralick_df = pd.DataFrame(haralick, columns=[
    'ASM', 'Contrast', 'Correlation', 'Variance', 'IDM', 'Sum_Average',
    'Sum_Variance', 'Sum_Entropy', 'Entropy', 'Difference_Variance',
    'Difference_Entropy', 'IMC1', 'IMC2'
])

# 데이터프레임 출력
print(haralick_df)

           ASM     Contrast  Correlation     Variance       IDM  Sum_Average  \
0     0.000609   646.281536     0.889691  2929.366619  0.197690   188.516126   
1     0.000986   753.491949     0.947918  7232.684350  0.230113   301.534133   
2     0.000611   573.337685     0.939569  4743.964206  0.175806   277.066613   
3     0.011206   132.208872     0.886605   582.616944  0.485988    54.347856   
4     0.002986   457.923017     0.931123  3324.605567  0.317951   227.930042   
...        ...          ...          ...          ...       ...          ...   
6329  0.000398   231.210898     0.979381  5606.753123  0.247869   269.102102   
6330  0.000531   718.349006     0.849774  2388.876832  0.157800   250.971430   
6331  0.001080   174.553447     0.984492  5627.147242  0.270758   236.624732   
6332  0.000324  1086.098211     0.852492  3681.542436  0.164291   302.343720   
6333  0.000961   273.519083     0.967100  4155.920027  0.315945   301.266977   

      Sum_Variance  Sum_Entropy    Entr

In [20]:
tmp = pd.concat([df, haralick_df], axis=1)
tmp = tmp.sort_values(by='score', ascending=False).reset_index(drop=True)

In [25]:
numeric_cols = tmp.select_dtypes(include=['float64', 'int64']).columns
numeric_df = tmp[numeric_cols]
numeric_df

,score,width,height,bbox_size,x,y,entropy,laplacian,brightness,B_sd,...,Variance,IDM,Sum_Average,Sum_Variance,Sum_Entropy,Entropy,Difference_Variance,Difference_Entropy,IMC1,IMC2
0,0.960156,175.78,365.92,64321.4176,231.39,23.32,6.991802,2396.599312,109.016344,55.437876,...,3085.183045,0.272328,218.321967,11805.860404,7.976299,11.780171,0.000351,4.483578,-0.316311,0.993586
1,0.946107,175.98,269.79,47477.6442,222.64,85.72,7.640442,4402.490432,136.528625,57.042084,...,3246.922688,0.086307,272.560296,12048.803662,8.617964,13.766669,0.000097,5.728885,-0.197973,0.974561
2,0.945302,247.26,443.74,109719.1524,39.11,48.82,7.261376,1511.846484,138.665786,52.863906,...,2796.926974,0.255842,277.354480,10746.326177,8.235842,12.028241,0.000368,4.271601,-0.343483,0.996333
3,0.942805,125.15,386.46,48365.4690,195.29,211.79,7.541892,7403.016087,98.924808,55.624675,...,3097.140189,0.101048,198.059275,11180.774590,8.463239,13.577417,0.000113,5.674594,-0.199670,0.974415
4,0.941582,131.46,228.37,30021.5202,274.43,78.75,7.315833,3676.558615,153.935550,39.944085,...,1602.795150,0.084901,307.765087,5720.189042,8.223095,13.280933,0.000107,5.537443,-0.185451,0.964452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,0.213297,47.54,96.43,4584.2822,406.07,0.00,6.181008,663.084400,61.003103,33.619329,...,1128.807063,0.194234,123.457601,4328.662984,7.114964,10.553525,0.000480,4.492450,-0.296937,0.986859
6330,0.212928,80.53,135.90,10944.0270,0.00,0.00,4.793607,40.994660,35.096204,63.665830,...,3945.478955,0.519975,68.458457,15714.912591,5.674219,7.333457,0.001055,2.768688,-0.457672,0.993321
6331,0.201631,69.67,65.00,4528.5500,408.33,362.50,7.348773,3286.761896,97.960981,43.993721,...,1928.911575,0.099488,196.412494,7113.236356,8.251805,12.176833,0.000166,5.272366,-0.342831,0.996724
6332,0.000000,54.45,68.97,3755.4165,228.50,91.80,5.477101,498.482781,22.003268,14.538881,...,207.225784,0.230318,44.017930,739.038855,6.378352,9.595479,0.000745,3.851126,-0.246428,0.964953


In [44]:
tmp2 = numeric_df.drop(columns=['x','y'])
tmp2

,score,width,height,bbox_size,entropy,laplacian,brightness,B_sd,red,blue,...,Variance,IDM,Sum_Average,Sum_Variance,Sum_Entropy,Entropy,Difference_Variance,Difference_Entropy,IMC1,IMC2
0,0.960156,175.78,365.92,64321.4176,6.991802,2396.599312,109.016344,55.437876,89.456751,116.950716,...,3085.183045,0.272328,218.321967,11805.860404,7.976299,11.780171,0.000351,4.483578,-0.316311,0.993586
1,0.946107,175.98,269.79,47477.6442,7.640442,4402.490432,136.528625,57.042084,143.867764,132.133001,...,3246.922688,0.086307,272.560296,12048.803662,8.617964,13.766669,0.000097,5.728885,-0.197973,0.974561
2,0.945302,247.26,443.74,109719.1524,7.261376,1511.846484,138.665786,52.863906,129.274911,145.894198,...,2796.926974,0.255842,277.354480,10746.326177,8.235842,12.028241,0.000368,4.271601,-0.343483,0.996333
3,0.942805,125.15,386.46,48365.4690,7.541892,7403.016087,98.924808,55.624675,94.581389,78.445679,...,3097.140189,0.101048,198.059275,11180.774590,8.463239,13.577417,0.000113,5.674594,-0.199670,0.974415
4,0.941582,131.46,228.37,30021.5202,7.315833,3676.558615,153.935550,39.944085,155.740893,157.968963,...,1602.795150,0.084901,307.765087,5720.189042,8.223095,13.280933,0.000107,5.537443,-0.185451,0.964452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,0.213297,47.54,96.43,4584.2822,6.181008,663.084400,61.003103,33.619329,63.007535,65.351950,...,1128.807063,0.194234,123.457601,4328.662984,7.114964,10.553525,0.000480,4.492450,-0.296937,0.986859
6330,0.212928,80.53,135.90,10944.0270,4.793607,40.994660,35.096204,63.665830,31.542130,40.154444,...,3945.478955,0.519975,68.458457,15714.912591,5.674219,7.333457,0.001055,2.768688,-0.457672,0.993321
6331,0.201631,69.67,65.00,4528.5500,7.348773,3286.761896,97.960981,43.993721,106.094537,103.439242,...,1928.911575,0.099488,196.412494,7113.236356,8.251805,12.176833,0.000166,5.272366,-0.342831,0.996724
6332,0.000000,54.45,68.97,3755.4165,5.477101,498.482781,22.003268,14.538881,27.245915,15.398148,...,207.225784,0.230318,44.017930,739.038855,6.378352,9.595479,0.000745,3.851126,-0.246428,0.964953


In [47]:
not_normalized_df = pd.concat([tmp['image_file'], tmp2], axis=1)
# not_normalized_df.to_csv("df.csv", index=False)


In [58]:
numeric_cols = tmp.select_dtypes(include=['float64', 'int64']).columns
numeric_df = tmp[numeric_cols]
df_normalized = (numeric_df - numeric_df.min()) / (numeric_df.max() - numeric_df.min())

In [61]:
df_normalized.columns

Index(['score', 'width', 'height', 'bbox_size', 'x', 'y', 'entropy',
       'laplacian', 'brightness', 'B_sd', 'red', 'blue', 'green', 'color_var',
       'contrast', 'img_size', 'edge_intensity', 'saturation', 'sift',
       'c_moment_x', 'c_moment_y', 'bbox_prop', 'ASM', 'Contrast',
       'Correlation', 'Variance', 'IDM', 'Sum_Average', 'Sum_Variance',
       'Sum_Entropy', 'Entropy', 'Difference_Variance', 'Difference_Entropy',
       'IMC1', 'IMC2'],
      dtype='object')

In [60]:
pd.concat([tmp['image_file'], df_normalized], axis=1).to_csv("add_haralick.csv", index=False)


- Angular Second Moment (ASM): 에지 강도의 정도를 나타내며, 값이 작을수록 텍스처가 덜 일정하거나 변화가 많음을 나타냅니다.

- Contrast: 인접한 픽셀 값의 대비를 나타냅니다. 값이 클수록 이미지의 대비가 뚜렷하고 선명함을 의미합니다.

- Correlation: 이미지에서 한 방향의 밝기 변화가 다른 방향으로 얼마나 유사한지를 나타내며, 1에 가까울수록 밝기 변화가 일정합니다.

- Sum of Squares (Variance): 픽셀 값의 분산을 나타냅니다. 값이 클수록 이미지의 텍스처가 다양하게 분포되어 있음을 나타냅니다.

- Inverse Difference Moment (IDM): 픽셀 값의 변화가 얼마나 부드럽게 일어나는지를 나타내며, 값이 클수록 부드러운 텍스처를 의미합니다.

- Sum Average: 텍스처에서 발생한 픽셀 값의 합의 평균을 나타냅니다.

- Sum Variance: 픽셀 값의 합의 분산을 나타냅니다.

- Sum Entropy: 텍스처의 복잡성을 나타내며, 값이 클수록 텍스처가 복잡하다는 것을 의미합니다.

- Entropy: 이미지의 무질서도를 나타냅니다. 값이 클수록 이미지가 무질서하다는 것을 의미합니다.

- Difference Variance: 픽셀 값 간의 차이의 분산을 나타냅니다.

- Difference Entropy: 픽셀 값 간의 차이의 무질서도를 나타냅니다.

- Information Measures of Correlation 1 (IMC1): Correlation에 기반한 정보의 양을 나타냅니다.

- Information Measures of Correlation 2 (IMC2): Correlation에 기반한 정보의 양을 나타냅니다.
